In [ ]:
!pip install umap

  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=c75a93d42467a8f771a7e4716acb72790e0ce1d569e6d7810eb05d9aaaa08db5
  Stored in directory: /root/.cache/pip/wheels/65/55/85/945cfb3d67373767e4dc3e9629300a926edde52633df4f0efe
Successfully built umap


In [ ]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import math
from sklearn.cluster import dbscan
from sklearn.cluster import KMeans
from sklearn.cluster import OPTICS
from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.base import ClusterMixin
from sklearn.utils import check_array
import matplotlib.pyplot as plt
import umap

csvs = ['prag_dates.csv', 'dates.csv', 'prag.csv', 'res.csv', 'ds_clumps.csv', 'ideology.csv', 'year.csv']

df=pd.read_csv('./Newsbooks_Intermediary_Data/res.csv')

features = ['AcademicTerms',
       'AcademicWritingMoves', 'CharacterCognitiveAgency',
       'CharacterCognitiveStates', 'CharacterDialogueCues',
       'CharacterDisclosive', 'CharacterPersonNamesPronouns',
       'CharacterSubjective', 'CharacterTraitsTypes', 'CitationAuthority',
       'CitationControversy', 'CitationGeneral', 'CitationHedged',
       'CitationNegative', 'CitationNeutral', 'CitationSpeakerLookMood',
       'ConfidenceHedged', 'ConfidenceHigh', 'ConfidenceHighNegative',
       'ConfidenceLow', 'Contingent', 'Description', 'Facilitate',
       'FirstPerson', 'ForceStressed', 'Future', 'InformationChange',
       'InformationChangeNegative', 'InformationChangePositive',
       'InformationExposition', 'InformationPlace', 'InformationReportVerbs',
       'InformationStates', 'InformationTopics', 'Inquiry', 'Interactive',
       'MetadiscourseCohesive', 'MetadiscourseInteractive', 'NarrativeActs',
       'NarrativeBiographicalTime', 'NarrativeDuration',
       'NarrativeExpectation', 'NarrativeHurriedTime', 'NarrativeImmediacy',
       'NarrativePast', 'NarrativeSceneShifts', 'NarrativeSequence',
       'NarrativeSimultaneity', 'NarrativeTimeReference',
       'NarrativeTimeShifts', 'NarrativeTrends', 'NarrativeTurningPoint',
       'NarrativeVerbs', 'Negative', 'Orphaned', 'Positive', 'PublicTerms',
       'Reasoning', 'Responsibility', 'Strategic', 'SyntacticComplexity',
       'Uncertainty', 'Updates']




FileNotFoundError: ignored

complete

In [ ]:
X = df[features]


tsne_embedded = TSNE(n_components=2, metric='euclidean').fit_transform(X)
pca_embedded = PCA(n_components=2).fit_transform(X)

TSNE_df = pd.DataFrame(tsne_embedded)
pca_df = pd.DataFrame(pca_embedded)

reducer=umap.UMAP()
umap_embedding=reducer.fit_transform(X) # takes 1-10 seconds
umap_df=pd.DataFrame(umap_embedding)

In [ ]:
clstr_dbscan_tsne=dbscan(tsne_embedded,eps=3.3) 
pd.DataFrame(clstr_dbscan_tsne[1]).describe()
clstr_optics_tsne=OPTICS(min_samples=.1).fit_predict(tsne_embedded)
plt.scatter(TSNE_df[0],TSNE_df[1],c=clstr_dbscan_tsne[1])
plt.show()
plt.scatter(TSNE_df[0],TSNE_df[1],c=clstr_optics_tsne)
plt.show()
plt.scatter(pca_df[0],pca_df[1])



> helpers




In [ ]:
def dfNormalizer(df):
    xmin=np.array(df[0]).min()
    xdif=np.array(df[0]).max()-xmin
    ymin=np.array(df[1]).min()
    ydif=np.array(df[1]).max()-ymin
    resx=[]
    resy=[]
    for xelement in df[0]:
        xelement=(xelement - xmin)/xdif
        resx.append(xelement)

    for yelement in df[1]:
        yelement=(yelement - ymin)/ydif
        resy.append(yelement)

    res=pd.DataFrame(zip(resx,resy))

    return res

def pdNormalizer(ndarray):
    return dfNormalizer(pd.DataFrame(ndarray)).to_numpy()
def showplot(ndarr,cluster):
    plt.scatter(pd.DataFrame(ndarr)[0],pd.DataFrame(ndarr)[1],c=cluster)
    plt.show()
def createz(title):
       res=[]
       arr=df[title]

       mu= np.mean(arr)
       sigma=np.std(arr)
       for item in arr:
              z= abs( (item - mu)/sigma)
              res.append(z)
       return np.array(res)

In [ ]:
def findForgeries(newsbookstitle:str):
       print(f"TITLE:{newsbookstitle}")
       nb=df[df["Newsbooks Title"].str.match(newsbookstitle)][features]
       names=pd.DataFrame(df[df["Newsbooks Title"].str.match(newsbookstitle)]["text_name"])
       print(nb.shape)
       if nb.shape[0]<5:
              return
       
       nbtsne=TSNE(n_components=2, metric='euclidean').fit_transform(nb)
       nbpca = PCA(n_components=2).fit_transform(nb)
       nbumap = umap.UMAP().fit_transform(nb)

       normalized_tsne=dfNormalizer(nbtsne)
       normalized_pca=dfNormalizer(nbpca)
       normalized_umap=dfNormalizer(nbumap)

       
       optics_tsne=OPTICS(min_samples=.5).fit_predict(nbtsne)
       optics_pca=OPTICS(min_samples=.5).fit_predict(nbpca)
       # optics_umap=OPTICS(min_samples=.5).fit_predict(nbumap)
       
       dbscan_pca=dbscan(nbpca,eps=.5)
      
       showplot(nbtsne,optics_tsne)

       showplot(nbpca,optics_pca)
       showplot(nbumap,optics_umap)
       showplot(nbpca,dbscan_pca[1])

      #   names["tnse"]=list(nbtsne)
      #  names["pca"]=list(nbpca)
      #  print(names)

       # showplot(normalized_tsne,optics_tsne)
       # showplot(normalized_pca,optics_pca)
       # showplot(normalized_umap,optics_umap)
       # showplot(normalized_pca,dbscan)

       return

In [ ]:
sampletitle="The kingdomes vveekly intelligencer"
sampletitle2="Mercurius pragmaticus"
findForgeries(sampletitle)

In [ ]:
for name in df["Newsbooks Title"].unique():
    findForgeries(name)

title	confidence	text name	reason	notes:	
mercurius elencticus	slight	43_clean	pca	all of the other "mercurius electencus" have text names 555-584	
mercurius elencticus	slight	44_clean	pca		
mercurius elencticus	slight	45_clean	pca		
the kingdomes faithful inpartial	slight	72_clean	pca		
The kingdomes vveekly intelligencer	medium	188_clean	pca	same sit as elenctucus	
The kingdomes vveekly intelligencer	medium	189_clean	pca		
The kingdomes vveekly intelligencer	slight	216_clean	pca		
The man in the moon	high	238_clean	tsne		
The moderate intelligencer	slight	270_clean	pca		
A briefe relation of some affaires	high	308_clean	pca		
A briefe relation of some affaires	medium	320_clean	pca		
Perfect occurrences of every dayes	medium	340_clean	pca	same sit	
Perfect occurrences of every dayes	medium	341_clean	pca		
Severall proceedings in Parliament	high	389_clean	pca		
A perfect summary of exact passages	medium	396_clean	pca		
A perfect diurnall of some passages	slight	511_clean	pca		
A perfect diurnall of some passages	slight	512_clean	pca		